# Camera Calibration
## Code cells 1-2

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        write_name = './output_images/corners_found'+str(idx)+'.jpg'
        cv2.imwrite(write_name, img)
        cv2.imshow('img', img)
        cv2.waitKey(500)
cv2.destroyAllWindows()

In [ ]:
import pickle
%matplotlib inline

# Test undistortion on an image
load_img='calibration1' #change this to image name
img = cv2.imread('./camera_cal/'+load_img+'.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('./output_images/undistort_'+load_img+'.jpg',dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "dist_pickle.p", "wb" ) )
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

# Apply Thresholding
## Code cells 3

In [ ]:
import matplotlib.image as mpimg

image = mpimg.imread('./test_images/test4.jpg')
# undistort image
dst = cv2.undistort(image, mtx, dist, None, mtx)

def pipeline(img, s_thresh=(170, 255), sx_thresh=(30, 200), kernelsize=7):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, 1)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

result = pipeline(dst)
combined_binary = np.zeros_like(result[:,:,0])
combined_binary[(result[:,:,1] == 1) | (result[:,:,2] == 1)] = 1

# Plot the result
f, axes = plt.subplots(2, 2, figsize=(24, 18))
f.tight_layout()

axes=axes.ravel()
axes[0].imshow(dst)
axes[0].set_title('Original Image', fontsize=40)

axes[1].imshow(result[:,:,1], cmap='gray')
axes[1].set_title('X gradient threshold', fontsize=40)

axes[2].imshow(result[:,:,2], cmap='gray')
axes[2].set_title('S channel threshold', fontsize=40)

axes[3].imshow(combined_binary, cmap='gray')
axes[3].set_title('Combined threshold', fontsize=40)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Test Threshold Values
## Code cells 4-6

In [ ]:
### Test gradx low thresh value
read_image='test4'
image = mpimg.imread('./test_images/'+read_image+'.jpg')
# undistort image
dst = cv2.undistort(image, mtx, dist, None, mtx)

def pipeline(img, s_thresh, sx_thresh, kernelsize=3):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary

low_thresh=np.linspace(0,70,15)
s_thresh=(170, 255)

# Plot the result
f, axes = plt.subplots(5, 3, figsize=(50, 50))
#plt.suptitle(read_image, fontsize=60)
f.tight_layout()
axes=axes.ravel()

for x in range(0, len(low_thresh)):
    sx_thresh=(low_thresh[x], 200)
    result = pipeline(dst, s_thresh, sx_thresh)

    axes[x].imshow(result[:,:,1], cmap='gray')
    axes[x].set_title('Lower gradx threshold: '+str(low_thresh[x]), fontsize=40)


In [ ]:
### Test gradx kernel size
read_image='test4'
image = mpimg.imread('./test_images/'+read_image+'.jpg')
# undistort image
dst = cv2.undistort(image, mtx, dist, None, mtx)

def pipeline(img, s_thresh, sx_thresh, kernelsize):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary

ksize=np.arange(3,33,2)

# Plot the result
f, axes = plt.subplots(5, 3, figsize=(50, 50))
#plt.suptitle(read_image, fontsize=60)
f.tight_layout()
axes=axes.ravel()

for x in range(0, len(ksize)):
    sx_thresh=(low_thresh[x], 200)
    result = pipeline(dst, s_thresh=(170, 255), sx_thresh=(30, 200), kernelsize=ksize[x])

    axes[x].imshow(result[:,:,1], cmap='gray')
    axes[x].set_title('Gradx kernel size: '+str(ksize[x]), fontsize=40)


In [ ]:
### Test s channel low threshold
read_image='test4'
image = mpimg.imread('./test_images/'+read_image+'.jpg')
# undistort image
dst = cv2.undistort(image, mtx, dist, None, mtx)

def pipeline(img, s_thresh, sx_thresh, kernelsize):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary

low_thresh=np.arange(90,240,10)

# Plot the result
f, axes = plt.subplots(5, 3, figsize=(50, 50))
#plt.suptitle(read_image, fontsize=60)
f.tight_layout()
axes=axes.ravel()

for x in range(0, len(low_thresh)):
    sx_thresh=(low_thresh[x], 200)
    result = pipeline(dst, s_thresh=(low_thresh[x], 255), sx_thresh=(30, 200), kernelsize=7)

    axes[x].imshow(result[:,:,2], cmap='gray')
    axes[x].set_title('S channel low thresh: '+str(low_thresh[x]), fontsize=40)


# Perspective Transform
## Code cells 7-8

In [ ]:
### Open straight line image, draw bounding rectangle, and perform perspective transform

from PIL import Image, ImageDraw

im = np.asarray(Image.open("./test_images/straight_lines1.jpg"))
#im = region_of_interest(im, vertices)

dist_pickle=pickle.load(open("dist_pickle.p", "rb"))
mtx=dist_pickle["mtx"]
dist=dist_pickle["dist"]

src = np.float32([[236,690],[562,470],[718,470],[1070,690]])
dst = np.float32([[300,720],[300,50],[980,50],[980,720]])

undistorted = cv2.undistort(im, mtx, dist, None, mtx)
undistorted = Image.fromarray(np.uint8(undistorted))

draw = ImageDraw.Draw(undistorted)
draw.polygon(src, fill=None, outline=128)
#plt.imshow(undistorted)

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
warped = cv2.warpPerspective(np.asarray(undistorted), M, im.shape[1::-1], flags=cv2.INTER_LINEAR)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
ax1.imshow(undistorted)
ax1.set_title('Distortion Corrected', fontsize=12)
ax2.imshow(warped)
ax2.set_title('Overhead Perspective', fontsize=12)

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline

image = mpimg.imread('./test_images/test4.jpg')
left_line, right_line = Line(), Line() # initialize line objects to keep track of left and right lane lines

def pipeline(img, s_thresh=(170, 255), sx_thresh=(30, 200), kernelsize=7):
    img = np.copy(img)
    # undistort image
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    combined_binary = region_of_interest(combined_binary, [vertices])
    warped = cv2.warpPerspective(np.asarray(combined_binary), M, img.shape[1::-1], flags=cv2.INTER_LINEAR)

    return combined_binary, warped

result = pipeline(image)

# Read in a thresholded image
warped = result[1]
# window settings
window_width = 50 
window_height = 120 # Break image into 6 vertical layers since image height is 720
margin = 50 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),
           max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin, left_line, right_line):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    left_line.recent_xfitted.append(l_center)
    right_line.recent_xfitted.append(r_center)
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], 
                             axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # If that layer is empty
        if np.argmax(conv_signal[l_min_index:l_max_index]) == 0:
            l_center = l_center+margin
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # If that layer is empty
        if np.argmax(conv_signal[r_min_index:r_max_index]) == 0:
            r_center = r_center+margin
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
        
        left_line.recent_xfitted.append(l_center)
        right_line.recent_xfitted.append(r_center)

    return window_centroids

window_centroids = find_window_centroids(warped, window_width, window_height, margin, left_line, right_line)

# If we found any window centers
if len(window_centroids) > 0:

    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # Go through each level and draw the windows 	
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# If no window centers found, just display orginal road image
else:
    output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

y_coords=np.arange(720,0,-window_height)-60
left_x=np.asarray(window_centroids)[:,0]
left_fit=np.polyfit(y_coords,left_x,2)

right_x=np.asarray(window_centroids)[:,1]
right_fit=np.polyfit(y_coords,right_x,2)

# Generate x and y values for plotting
ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Plot the result
f, axes = plt.subplots(2, 2, figsize=(24, 18))
f.tight_layout()

axes=axes.ravel()
axes[0].imshow(image)
axes[0].set_title('Original Image', fontsize=40)

axes[1].imshow(result[0], cmap='gray')
axes[1].set_title('Threshold Image', fontsize=40)

axes[2].imshow(result[1], cmap='gray')
axes[2].set_title('Perspective Image', fontsize=40)

axes[3].imshow(output, cmap='gray')
axes[3].set_title('Window Fitting Results', fontsize=40)
axes[3].plot(left_fitx, ploty, color='red')
axes[3].plot(right_fitx, ploty, color='red')

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Create an image to draw the lines on
warp_zero = np.zeros_like(warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))
#pts = np.squeeze(pts)

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
plt.figure()
plt.imshow(result)

# Test on Video
## Code cells 9-14

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average first x values of the fitted line over the last n iterations
        self.bestx = None   
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

In [ ]:
### Packages and variables

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import statistics

dist_pickle=pickle.load(open("dist_pickle.p", "rb"))
mtx=dist_pickle["mtx"]
dist=dist_pickle["dist"]

# Line objects
left_line, right_line = Line(), Line()

# Mask layer
vertices = np.array([[1,690],[590,440],[700,440],[1270,690]], dtype=np.int32)
# Transform variables
src = np.float32([[236,690],[562,470],[718,470],[1070,690]])
dst = np.float32([[300,720],[300,50],[980,50],[980,720]])

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

In [ ]:
### Subfunctions for pipeline

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, 1)
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

def threshold_img(img, s_thresh=(170, 255), sx_thresh=(30, 200), kernelsize=7):
    # Pass in camera calibrated image. This function does not run camera undistortion
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0, ksize=kernelsize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    combined_binary = region_of_interest(combined_binary, [vertices])
    warped = cv2.warpPerspective(np.asarray(combined_binary), M, img.shape[1::-1], flags=cv2.INTER_LINEAR)

    return warped

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),
           max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    
    return output

def find_window_centroids(image, window_width, window_height, margin, left_line, right_line):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    if left_line.detected == False: 
        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
            
    else:
        # Use previous detected starting point
        left_line.bestx = statistics.median(left_line.recent_xfitted[0:-1:6])
        right_line.bestx = statistics.median(right_line.recent_xfitted[0:-1:6])
        
        l_sum = np.sum(image[int(3*image.shape[0]/4):,int(left_line.bestx-2*margin):int(left_line.bestx+2*margin)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2+int(left_line.bestx-2*margin)
        r_sum = np.sum(image[int(3*image.shape[0]/4):,int(right_line.bestx-2*margin):int(right_line.bestx+2*margin)], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(right_line.bestx-2*margin)
        
    if len(left_line.recent_xfitted)>=60: # delete first entry if list contains 10 frames
        del left_line.recent_xfitted[0:6]
        del right_line.recent_xfitted[0:6]
        left_line.detected = True
        
    left_line.recent_xfitted.append(l_center)
    right_line.recent_xfitted.append(r_center)
        
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], 
                             axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # If that layer is empty
        if np.argmax(conv_signal[l_min_index:l_max_index]) == 0:
            l_center = l_center+margin
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # If that layer is empty
        if np.argmax(conv_signal[r_min_index:r_max_index]) == 0:
            r_center = r_center+margin
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))
        # Append to line objects
        left_line.recent_xfitted.append(l_center)
        right_line.recent_xfitted.append(r_center)
        
    return window_centroids

def draw_windows(warped, window_centroids, window_width, window_height, margin):
    # Pass in warped image
    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

    return output

def draw_lanes(image, ploty, left_fitx, right_fitx, curverad, deviation):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(image).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    #pts = np.squeeze(pts)

    # Draw the lane onto the warped blank image
    cv2.fillPoly(warp_zero, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(warp_zero, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    # Write radius of curvature
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result,'Curvature: '+str(int(curverad))+'m',(10,50), font, 1, (0,255,0),2)
    cv2.putText(result,'Centering: '+str(int(deviation*100))+'cm',(10,75), font, 1, (0,255,0),2)
    return result

In [ ]:
### Image pipeline

def process_image(image, left_line=left_line, right_line=right_line):
    image = cv2.undistort(image, mtx, dist, None, mtx)
    warped = threshold_img(image)
    
    # window settings
    window_width = 50 
    window_height = 120 # Break image into 6 vertical layers since image height is 720
    margin = 50 # How much to slide left and right for searching

    window_centroids = find_window_centroids(warped, window_width, window_height, margin, left_line, right_line)
    output = draw_windows(warped, window_centroids, window_width, window_height, margin)
    
    #print(left_line.recent_xfitted)
    
    y_coords=np.arange(720,0,-window_height)-60
    left_x=np.asarray(window_centroids)[:,0]
    left_fit=np.polyfit(y_coords,left_x,2)
    right_x=np.asarray(window_centroids)[:,1]
    right_fit=np.polyfit(y_coords,right_x,2)

    left_line.current_fit = left_fit
    right_line.current_fit = right_fit
    
    # Reshape line fitted values
    left_reshape=np.reshape(np.asarray(left_line.recent_xfitted),(-1,6))
    right_reshape=np.reshape(np.asarray(right_line.recent_xfitted),(-1,6))
    left_avg = np.mean(left_reshape, axis=0)
    right_avg = np.mean(right_reshape, axis=0)
    left_line.best_fit=np.polyfit(y_coords,left_avg,2)
    right_line.best_fit=np.polyfit(y_coords,right_avg,2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    #left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    #right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Generate x and y values for plotting using running average
    left_fitx = left_line.best_fit[0]*ploty**2 + left_line.best_fit[1]*ploty + left_line.best_fit[2]
    right_fitx = right_line.best_fit[0]*ploty**2 + right_line.best_fit[1]*ploty + right_line.best_fit[2]
    
    y_eval = np.max(ploty)
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    curverad=(left_curverad+right_curverad)/2
    
    # Calculate deviation from center of lane
    if left_line.bestx == None:
        deviation=0
    else:
        center=(left_line.bestx+right_line.bestx)/2 # lane center
        deviation=640-center # assume pixel 640 is the center of the car. positive value is deviation to the right
        deviation=deviation*xm_per_pix # convert to meters
    
    result = draw_lanes(image, ploty, left_fitx, right_fitx, curverad, deviation)
    return result

In [ ]:
# Import everything needed to edit/save/watch video clips
import imageio
from moviepy.editor import VideoFileClip
from IPython.display import HTML

clip_output='clip_output.mp4'
clip_input=VideoFileClip('project_video.mp4')

clip=clip_input.fl_image(process_image)
%time clip.write_videofile(clip_output, audio=False)

In [ ]:
image = mpimg.imread('./test_images/test4.jpg')
result = process_image(image)

plt.figure(figsize = (20,6))
plt.imshow(result)